# ERA5 mean 2m temperature

## Import packages

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
# Climatology periods
clima_start = ["1940-01", "1941-01", "1981-01"]
clima_stop = ["2022-12", "1980-12", "2020-12"]
assert len(clima_start) == len(clima_stop) == 3

## Define request

In [ ]:
collection_id = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": "2m_temperature",
    "time": "00:00",
    "format": "grib",
}

# Parameters to speed up I/O
open_mfdataset_kwargs = {
    "concat_dim": "forecast_reference_time",
    "combine": "nested",
    "data_vars": "minimal",
    "coords": "minimal",
    "compat": "override",
    "parallel": True,
}

## Download and transform

In [ ]:
clima_datasets = []
month_datasets = []
season_datasets = []
for start, stop in zip(clima_start, clima_stop):
    requests = download.update_request_date(
        request, start=start, stop=stop, stringify_dates=True
    )
    for transform_func, ds_list in zip(
        (
            diagnostics.time_weighted_mean,
            diagnostics.monthly_weighted_mean,
            diagnostics.seasonal_weighted_mean,
        ),
        (clima_datasets, month_datasets, season_datasets),
    ):
        print(f"{start=} {stop=} {transform_func.__name__}")
        ds = download.download_and_transform(
            collection_id,
            requests,
            chunks={"year": 1, "month": 1},
            transform_chunks=False,
            transform_func=transform_func,
            transform_func_kwargs={"weights": False},
            **open_mfdataset_kwargs,
        )
        ds = ds.drop(["realization", "leadtime", "surface", "time"])
        ds_list.append(ds.expand_dims(period=[f"({start}, {stop})"]))
clima_ds = xr.concat(clima_datasets, "period")
month_ds = xr.concat(month_datasets, "period")
season_ds = xr.concat(season_datasets, "period")

## Plot climatology maps

In [ ]:
for ds in (clima_ds, season_ds):
    da = ds["t2m"].isel(period=0)
    plot.projected_map(
        da,
        projection=ccrs.Robinson(),
        cmap="YlOrRd",
        levels=9,
        robust=True,
        extend="both",
        col="season" if "season" in da.dims else None,
        col_wrap=2,
    )
    plt.show()

## Plot anomaly maps

In [ ]:
for ds in (clima_ds, season_ds):
    with xr.set_options(keep_attrs=True):
        da = ds["t2m"].drop_isel(period=0).diff("period")
    da.attrs["long_name"] += " anomaly"
    da["period"] = [" — ".join(ds["period"].values.tolist()[::-1][:2])]
    plot.projected_map(
        da,
        projection=ccrs.Robinson(),
        cmap="RdBu_r",
        levels=9,
        center=0,
        robust=True,
        extend="both",
        col="season" if "season" in da.dims else None,
        col_wrap=2,
    )
    plt.show()

## Plot seasonal cycle

In [ ]:
da = diagnostics.spatial_weighted_mean(month_ds["t2m"])
da.plot(hue="period")
plt.grid()

# Show table with values in Celsius
(da - 273.15).round(1).to_pandas()